# Intro autolog mlflow

Links examples:

- sklearn: https://mlflow.org/docs/latest/python_api/mlflow.sklearn.html#mlflow.sklearn.autolog

- tensorflow: https://github.com/mlflow/mlflow/blob/master/examples/keras/train.py

- other examples: https://mlflow.org/docs/latest/tutorials-and-examples/index.html

In [ ]:
import mlflow
!pip show mlflow # version mflow used

### 1. Autolog tensorflow

In [ ]:
"""Trains and evaluate a simple MLP
on the Reuters newswire topic classification task.
"""
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import reuters
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer

# The following import and function call are the only additions to code required
# to automatically log metrics and parameters to MLflow.
import mlflow

mlflow.tensorflow.autolog()

max_words = 1000
batch_size = 32
epochs = 5

print("Loading data...")
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words, test_split=0.2)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")

num_classes = np.max(y_train) + 1
print(num_classes, "classes")

print("Vectorizing sequence data...")
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode="binary")
x_test = tokenizer.sequences_to_matrix(x_test, mode="binary")
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

print("Convert class vector to binary class matrix (for use with categorical_crossentropy)")
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

print("Building model...")
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(
    x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1
)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print("Test score:", score[0])
print("Test accuracy:", score[1])

### 2. Autolog sklearn
Note: the version of sklearn used is not soported

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

#mlflow.sklearn.autolog() # oprueba autolog
mlflow.sklearn.autolog()

if __name__ == "__main__":
    X = np.array([-2, -1, 0, 1, 2, 1, 10, 12, 23]).reshape(-1, 1)
    y = np.array([0, 0, 1, 1, 1, 0, 0,0,0])
    lr = LogisticRegression()

    lr.fit(X, y)
    score = lr.score(X, y)
    print(f"Score: {score}")
    #mlflow.log_metric("score", score)
    predictions = lr.predict(X)
    signature = infer_signature(X, predictions)
    mlflow.sklearn.log_model(lr, "model", signature=signature)
    print(f"Model saved in run {mlflow.active_run().info.run_uuid}")